In [2]:
!pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for autogluon from https://files.pythonhosted.org/packages/ce/7b/db15f5d33bb0bcdcaead1740ccb7018bc560a9fde14edd177c62f1051844/autogluon-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.core[all]==1.0.0 from https://files.pythonhosted.org/packages/d6/0b/d2d017b328a2101348c2a07752f57ae6f4c47bbb67e5527b10823f1db55b/autogluon.core-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.features==1.0.0 from https://files.pythonhosted.org/packages/a5/21/490e0a4310c7aaa22961c5885e2e03f0a6e8097d1d4af0f6c9b22d8fba50/autogluon.features-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.tabular[all]==1.0.0 from https://files.pythonhosted.org/packages/98/30/8576a70043614aa1137a229916010b8080374fe131aeeee6323a93189b88/autogluon.tabular-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information 

In [4]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

from fastparquet import ParquetFile
from utils import feature_drop

In [5]:
file_path = "Data/train_ai_comp_final_dp.parquet"
pf = ParquetFile(file_path)
df = pf.to_pandas()
df = feature_drop(df)
df.head()

/cephfs/soft/anaconda3/envs/students2/lib/python3.11/site-packages/IPython/core/formatters.py:344: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,id,target,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1761,1759,85,105469,191,46,0,6,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1761,1759,120,105610,144,71,0,135,...,0,0,0,0,0,0,0,0,0,0
2,2,0,890,1759,141,105227,191,11,0,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1599,966,30,102441,191,8,0,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1761,1759,85,104006,191,2,0,0,...,0,0,7642,0,0,0,0,0,0,7642


In [6]:
X = df.drop(columns=['target', "id"])
y = df['target']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
to_train = pd.concat([X_train, y_train], axis=1)

In [7]:
to_train

/cephfs/soft/anaconda3/envs/students2/lib/python3.11/site-packages/IPython/core/formatters.py:344: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076,target
187083,1761,1759,117,105345,191,152,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
585908,1761,1759,88,74109,191,6,125,290,298,176,...,0,0,0,0,0,0,0,0,0,0
241239,1630,601,105,104480,191,34,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175984,1639,17,86,102228,0,12,125,290,298,176,...,0,0,0,0,0,0,0,0,0,0
393146,1761,1759,32,82893,191,44,125,290,298,176,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346164,1633,1759,137,104483,18,38,0,0,211,0,...,4370,5471,2783,0,0,3107,3446,4370,5471,0
488156,1761,1759,104,104948,78,14,0,92,191,0,...,18153,47941,0,0,0,0,0,18153,47941,0
176273,1633,17,141,104320,191,67,0,177,57,0,...,17306,29677,0,0,0,0,0,17306,29677,0
196223,1761,1759,73,97814,93,3,0,0,79,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(to_train)
predictor = TabularPredictor(label="target", eval_metric="roc_auc").fit(train_data, presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels/ag-20240313_105628"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240313_105628/ds_sub_fit/sub_fit_ho.


Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20240313_105628/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Aug 25 17:23:54 UTC 2020
CPU Count:          64
Memory Avail:       156.13 GB / 251.58 GB (62.1%)
Disk Space Avail:   42012.65 GB / 66431.47 GB (63.2%)
Train Data Rows:    415691
Train Data Columns: 986
Label Column:       target
Problem Type:       binary
Preprocessing data ...
/cephfs/projects/ppashin/.local/lib/python3.11/site-packages/autogluon/tabular/learner/default_learner.py:189: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

In [9]:
to_test = pd.concat([X_test, y_test], axis=1)
test_data = TabularDataset(to_test)
predictor.leaderboard(test_data)

/cephfs/soft/anaconda3/envs/students2/lib/python3.11/site-packages/IPython/core/formatters.py:344: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/cephfs/soft/anaconda3/envs/students2/lib/python3.11/site-packages/IPython/core/formatters.py:344: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L2,0.749416,0.746930,roc_auc,82.493711,719.916124,1647.701530,6.310055,28.124857,514.239816,2,True,5
1,WeightedEnsemble_L3,0.748779,0.747233,roc_auc,82.512620,720.040953,1704.206433,0.018909,0.124829,56.504903,3,True,6
2,LightGBMXT_BAG_L1,0.746220,0.745775,roc_auc,7.349113,30.168069,814.278948,7.349113,30.168069,814.278948,1,True,2
3,WeightedEnsemble_L2,0.746134,0.745837,roc_auc,13.284452,59.005147,1137.943893,0.016418,0.114689,40.048369,2,True,4
4,LightGBM_BAG_L1,0.708247,0.702831,roc_auc,5.918921,28.722388,283.616576,5.918921,28.722388,283.616576,1,True,3
5,KNeighborsUnif_BAG_L1,0.546186,0.554798,roc_auc,62.915622,632.900809,35.566189,62.915622,632.900809,35.566189,1,True,1


In [ ]:
predictor.save('dir_save')